In [46]:
# 주피터 노트북에서 사용중인 가상환경 확인

import sys

if hasattr(sys, 'base_prefix'):
    print(f"base: {sys.base_prefix}")

if hasattr(sys, 'real_prefix'):
    print(f"real: {sys.real_prefix}")

if hasattr(sys, 'prefix'):
    print(f"prfx: {sys.prefix}")

base: /usr/local/Caskroom/miniconda/base/envs/xcda
prfx: /usr/local/Caskroom/miniconda/base/envs/xcda


In [47]:
# 이미지 리스트 불러오기

import os
from glob import glob

img_dir_path = "cv7ahand/training/rgb"
img_list = []
for path in glob(os.path.join(img_dir_path, '*jpg')):
    img_list.append(path)
#     print(path)

In [51]:
print(len(img_list))

mislabeled = []

with open('cv7ahand/annotations/mislabeled_1.txt', 'r') as f:
    for line in f.readlines():
        mislabeled.append(line.strip())

with open('cv7ahand/annotations/mislabeled_2.txt', 'r') as f:
    for line in f.readlines():
        mislabeled.append(line.strip())        
        
with open('cv7ahand/annotations/mislabeled_3.txt', 'r') as f:
    for line in f.readlines():
        mislabeled.append(line.strip())        

with open('cv7ahand/annotations/mislabeled_4.txt', 'r') as f:
    for line in f.readlines():
        mislabeled.append(line.strip())        
        
with open('cv7ahand/annotations/mislabeled_5.txt', 'r') as f:
    for line in f.readlines():
        mislabeled.append(line.strip())    
    
print(len(mislabeled))
# mislabeled

for i in img_list:
    j = list(map(str, i.split('/')))[-1]
    j = list(map(str, j.split('.')))[0] 
    if j in mislabeled:
        print(j)
        img_list.remove(i)
        
print(len(img_list))

4685
413
4685


In [52]:
# shuffle 후 train, val, test divide

import random

train_val_test_ratio = [8,1,1]

random.shuffle(img_list)

num_train = len(img_list) * train_val_test_ratio[0] // sum(train_val_test_ratio)
num_val = len(img_list) * train_val_test_ratio[1] // sum(train_val_test_ratio)
num_test = len(img_list) - num_train - num_val

print(len(img_list))
print(num_train, num_val, num_test)

img_list_train = img_list[:num_train]
img_list_val = img_list[num_train:num_train+num_val]
img_list_test = img_list[-num_test:]

print(len(img_list_train) + len(img_list_val) + len(img_list_test))
print(len(img_list_train), len(img_list_val), len(img_list_test))

4685
3748 468 469
4685
3748 468 469


In [53]:
# freihand dataset의 "categories" 부분 확인

# import json

# with open('freihand_val.json') as f:
#     frei = json.load(f)

# frei['categories']

In [54]:
if not os.path.exists("cv7ahand/annotations"):
    os.mkdir("cv7ahand/annotations")

## cv7ahand_train.json

In [55]:
import json

d = {}

d['info'] = {
    'description': 'CV7AHand', 
    'version': '1.0', 
    'year': '2022', 
    'date_created': '2022-06-01'
}

d['licenses'] = ""

d['images'] = []

d['annotations'] = []

d['categories'] = [{
    'supercategory': 'hand',
    'id': 1,
    'name': 'hand',
    'keypoints': [
        'wrist',
        'thumb1',
        'thumb2',
        'thumb3',
        'thumb4',
        'forefinger1',
        'forefinger2',
        'forefinger3',
        'forefinger4',
        'middle_finger1',
        'middle_finger2',
        'middle_finger3',
        'middle_finger4',
        'ring_finger1',
        'ring_finger2',
        'ring_finger3',
        'ring_finger4',
        'pinky_finger1',
        'pinky_finger2',
        'pinky_finger3',
        'pinky_finger4'],
    'skeleton': [
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [1, 6],
        [6, 7],
        [7, 8],
        [8, 9],
        [1, 10],
        [10, 11],
        [11, 12],
        [12, 13],
        [1, 14],
        [14, 15],
        [15, 16],
        [16, 17],
        [1, 18],
        [18, 19],
        [19, 20],
        [20, 21]]}]

In [56]:
# d_json = json.dumps(d, indent = 4, ensure_ascii=False) 

# print(d_json)

In [57]:
from tqdm import tqdm
import cv2
import mediapipe as mp

with mp.solutions.hands.Hands(model_complexity=1) as hands:
    for img_path in tqdm(img_list_train):


        img_info = {}

        file_name = img_path[9:]
        # print(file_name)
        img_info['file_name'] = file_name

        img = cv2.imread(img_path)
        # print(img.shape)
        img_info['height'] = img.shape[0]
        img_info['width'] = img.shape[1]

        img_id = list(map(str, img_path.split('/')))[-1]
        img_id = img_id.replace(".jpg", "")
        img_id = int(img_id)
        # print(img_id)
        img_info['id'] = img_id
        

        ann_info = {}

        ann_info['category_id'] = 1
        ann_info['id'] = img_id
        ann_info['image_id'] = img_id
        ann_info['iscrowd'] = 0

        ann_info['keypoints'] = []

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        kpts = hands.process(img)

        if not kpts.multi_hand_landmarks:
            continue
        else:
            for hand_landmarks in kpts.multi_hand_landmarks:
                for i in range(21):
                    x, y = hand_landmarks.landmark[i].x, hand_landmarks.landmark[i].y
                    x, y = x * img.shape[0], y * img.shape[1]
                    # print(x, y)
                    ann_info['keypoints'].append(x)
                    ann_info['keypoints'].append(y)
                    ann_info['keypoints'].append(1)
                break

        # ann_info['bbox']
        # ann_info['segmentation']
        # ann_info['area']
        d['annotations'].append(ann_info)    

        d['images'].append(img_info)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
100%|███████████████████████████████████████████████████████████| 3748/3748 [02:14<00:00, 27.91it/s]


In [58]:
# d_json = json.dumps(d, indent = 4, ensure_ascii=False) 

# print(d_json)

In [59]:
with open("cv7ahand/annotations/cv7ahand_train.json", "w", encoding='utf-8') as f:
    json.dump(d, f, indent=4, ensure_ascii=False)

## cv7ahand_val.json

In [60]:
import json

d = {}

d['info'] = {
    'description': 'CV7AHand', 
    'version': '1.0', 
    'year': '2022', 
    'date_created': '2022-06-01'
}

d['licenses'] = ""

d['images'] = []

d['annotations'] = []

d['categories'] = [{
    'supercategory': 'hand',
    'id': 1,
    'name': 'hand',
    'keypoints': [
        'wrist',
        'thumb1',
        'thumb2',
        'thumb3',
        'thumb4',
        'forefinger1',
        'forefinger2',
        'forefinger3',
        'forefinger4',
        'middle_finger1',
        'middle_finger2',
        'middle_finger3',
        'middle_finger4',
        'ring_finger1',
        'ring_finger2',
        'ring_finger3',
        'ring_finger4',
        'pinky_finger1',
        'pinky_finger2',
        'pinky_finger3',
        'pinky_finger4'],
    'skeleton': [
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [1, 6],
        [6, 7],
        [7, 8],
        [8, 9],
        [1, 10],
        [10, 11],
        [11, 12],
        [12, 13],
        [1, 14],
        [14, 15],
        [15, 16],
        [16, 17],
        [1, 18],
        [18, 19],
        [19, 20],
        [20, 21]]}]

In [61]:
from tqdm import tqdm
import cv2
import mediapipe as mp

with mp.solutions.hands.Hands(model_complexity=1) as hands:
    for img_path in tqdm(img_list_val):


        img_info = {}

        file_name = img_path[9:]
        # print(file_name)
        img_info['file_name'] = file_name

        img = cv2.imread(img_path)
        # print(img.shape)
        img_info['height'] = img.shape[0]
        img_info['width'] = img.shape[1]

        img_id = list(map(str, img_path.split('/')))[-1]
        img_id = img_id.replace(".jpg", "")
        img_id = int(img_id)
        # print(img_id)
        img_info['id'] = img_id
        

        ann_info = {}

        ann_info['category_id'] = 1
        ann_info['id'] = img_id
        ann_info['image_id'] = img_id
        ann_info['iscrowd'] = 0

        ann_info['keypoints'] = []

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        kpts = hands.process(img)

        if not kpts.multi_hand_landmarks:
            continue
        else:
            for hand_landmarks in kpts.multi_hand_landmarks:
                for i in range(21):
                    x, y = hand_landmarks.landmark[i].x, hand_landmarks.landmark[i].y
                    x, y = x * img.shape[0], y * img.shape[1]
                    # print(x, y)
                    ann_info['keypoints'].append(x)
                    ann_info['keypoints'].append(y)
                    ann_info['keypoints'].append(1)
                break

        # ann_info['bbox']
        # ann_info['segmentation']
        # ann_info['area']
        d['annotations'].append(ann_info)    

        d['images'].append(img_info)


100%|█████████████████████████████████████████████████████████████| 468/468 [00:16<00:00, 27.86it/s]


In [62]:
with open("cv7ahand/annotations/cv7ahand_val.json", "w", encoding='utf-8') as f:
    json.dump(d, f, indent=4, ensure_ascii=False)

## cv7ahand_test.json

In [66]:
import json

d = {}

d['info'] = {
    'description': 'CV7AHand', 
    'version': '1.0', 
    'year': '2022', 
    'date_created': '2022-06-01'
}

d['licenses'] = ""

d['images'] = []

d['annotations'] = []

d['categories'] = [{
    'supercategory': 'hand',
    'id': 1,
    'name': 'hand',
    'keypoints': [
        'wrist',
        'thumb1',
        'thumb2',
        'thumb3',
        'thumb4',
        'forefinger1',
        'forefinger2',
        'forefinger3',
        'forefinger4',
        'middle_finger1',
        'middle_finger2',
        'middle_finger3',
        'middle_finger4',
        'ring_finger1',
        'ring_finger2',
        'ring_finger3',
        'ring_finger4',
        'pinky_finger1',
        'pinky_finger2',
        'pinky_finger3',
        'pinky_finger4'],
    'skeleton': [
        [1, 2],
        [2, 3],
        [3, 4],
        [4, 5],
        [1, 6],
        [6, 7],
        [7, 8],
        [8, 9],
        [1, 10],
        [10, 11],
        [11, 12],
        [12, 13],
        [1, 14],
        [14, 15],
        [15, 16],
        [16, 17],
        [1, 18],
        [18, 19],
        [19, 20],
        [20, 21]]}]

In [67]:
from tqdm import tqdm
import cv2
import mediapipe as mp

with mp.solutions.hands.Hands(model_complexity=1) as hands:
    for img_path in tqdm(img_list_test):


        img_info = {}

        file_name = img_path[9:]
        # print(file_name)
        img_info['file_name'] = file_name

        img = cv2.imread(img_path)
        # print(img.shape)
        img_info['height'] = img.shape[0]
        img_info['width'] = img.shape[1]

        img_id = list(map(str, img_path.split('/')))[-1]
        img_id = img_id.replace(".jpg", "")
        img_id = int(img_id)
        # print(img_id)
        img_info['id'] = img_id
        

        ann_info = {}

        ann_info['category_id'] = 1
        ann_info['id'] = img_id
        ann_info['image_id'] = img_id
        ann_info['iscrowd'] = 0

        ann_info['keypoints'] = []

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img.flags.writeable = False
        kpts = hands.process(img)

        if not kpts.multi_hand_landmarks:
            continue
        else:
            for hand_landmarks in kpts.multi_hand_landmarks:
                for i in range(21):
                    x, y = hand_landmarks.landmark[i].x, hand_landmarks.landmark[i].y
                    x, y = x * img.shape[0], y * img.shape[1]
                    # print(x, y)
                    ann_info['keypoints'].append(x)
                    ann_info['keypoints'].append(y)
                    ann_info['keypoints'].append(1)
                break

        # ann_info['bbox']
        # ann_info['segmentation']
        # ann_info['area']
        d['annotations'].append(ann_info)    

        d['images'].append(img_info)


100%|█████████████████████████████████████████████████████████████| 469/469 [00:15<00:00, 30.16it/s]


In [68]:
with open("cv7ahand/annotations/cv7ahand_test.json", "w", encoding='utf-8') as f:
    json.dump(d, f, indent=4, ensure_ascii=False) 